In [ ]:
import plotly.express as px
import pandas as pd
import panel as pn
pn.extension('plotly', 'mathjax', sizing_mode="stretch_width")

px.set_mapbox_access_token("pk.eyJ1IjoiYXpvbG90aW4iLCJhIjoiY2xsdXpkY3hhMW5wNTNkcWlmenpkZ2Q1cCJ9.ITxYXNFB7_lnuNetDquy6w")

df = pd.read_csv("https://raw.githubusercontent.com/nitoloz/jupyter-notebooks/main/frankfurt-10k-trees/output_file.csv")

df['Common Name'] = df['Common Name'].replace(
    'Hybrid (for hybrids without a distinct common name)',
    'Hybrid'
)

# Replace 'output_file.csv' with the desired output file name
output_file = 'output_file.csv'

# Write the updated DataFrame back to a CSV file
df.to_csv(output_file, index=False)

df = df[df['PFLANZJAHR'] >= 1850]
min_value = df['PFLANZJAHR'].min()
max_value = df['PFLANZJAHR'].max()

In [ ]:
mapbox = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", size="KR_DURCHM",color="PFLANZJAHR", color_continuous_scale="Greens", range_color=(min_value, max_value), size_max=15, zoom=11, hover_data=["Common Name", "PFLANZJAHR", "KR_DURCHM", "BAUMHOEHE","STRASSE" ])
mapbox.update_traces(hovertemplate="<b>Name:</b> %{customdata[0]}<br><b>Year of planting:</b> %{customdata[1]}<br><b>Height and crown diameter</b> %{customdata[3]}m, %{customdata[2]}m<br><b>Address</b> %{customdata[4]}")

mapbox.update_layout(height=900)
mapbox

In [ ]:

def update_chart(values):
    fig = px.scatter_mapbox(df.loc[(df['PFLANZJAHR'] > values[0]) & (df['PFLANZJAHR'] < values[1])], lat="Latitude", lon="Longitude", size="KR_DURCHM",color="PFLANZJAHR", color_continuous_scale="Greens", range_color=(min_value, max_value), size_max=15, zoom=10)
    fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", size="KR_DURCHM",color="PFLANZJAHR", color_continuous_scale="Greens", range_color=(min_value, max_value), size_max=15, zoom=10)
    fig.update_layout(height=500)
    return fig;

# districts_area = pn.Column(int_range_slider, pn.bind(update_chart, int_range_slider))
# districts_area


In [ ]:
import plotly.graph_objects as go

year_counts = df['PFLANZJAHR'].value_counts().sort_index()
years_figure = go.Figure(data=go.Bar(x=year_counts.index, y=year_counts.values))

# Set the chart title and axis labels
years_figure.update_layout(title='Count of trees planted by Year',
                  xaxis_title='Year',
                  yaxis_title='Count of trees')
years_figure.update_layout(height=340)

years_figure.show()

In [ ]:
# Count the occurrences of each unique value in the 'GATTUNG' column
gattung_counts = df['Common Name'].value_counts().reset_index()
gattung_counts.columns = ['Common Name', 'Count']

# Create a bar chart using Plotly Express
species_figure = px.bar(gattung_counts, x='Common Name', y='Count', title='Total count of planted trees by species')
species_figure.update_xaxes(title='Species', tickangle=45)
species_figure.update_yaxes(title='Count')
species_figure.update_layout(height=340)
# Show the plot
species_figure.show()


In [ ]:
markdown = pn.pane.Markdown("""
# Frankfurt: A Green Metropolis

Frankfurt is to receive **10,000 new trees**, on streets, in squares, in green spaces - by 2030 [^1^].

It's undoubtedly a positive step that Frankfurt is set to welcome 10,000 new trees across its streets, squares, and green spaces by 2030, with an additional commitment to create three mini forests within the next two years. 
The proposed initiative, supported by the coalition of the Greens, SPD, FDP, and Volt, as well as the agreement from the CDU, demonstrates a collaborative effort towards environmental sustainability. 

However, amidst the celebration, it's crucial to consider the context provided by the Climate Decision Initiative members who point out that the 10,000 trees merely offset the losses incurred in the last five years. 

Although I wholeheartedly support this remarkable initiative, the article omits certain data. Presently, there are approximately 170,000 live trees in Frankfurt, all meticulously recorded in a tree cadastre. 
Analysis of this registry reveals that, over the past decade, nearly 1,000 trees have been planted annually. Notably, in the most robust years, the city witnessed the planting of up to 10,000 trees within a single year—examples include 9,447 trees in 1990, 11,867 in 1980, and 8,709 in 1970.

So keep it up, City of Frankfurt! Let's make our city greener together!
[^1^]: [10 000 Bäume für Frankfurt](https://www.fr.de/frankfurt/10-000-baeume-fuer-frankfurt-92325004.html)
""")
markdown

In [ ]:

template = pn.template.FastGridTemplate(
    site="Green Frankfurt", 
    title="Home of 170 000 trees", 
    prevent_collision=True, 
    cols={'lg': 12, 'md': 6, 'sm': 6, 'xs': 4, 'xxs': 2}, 
    row_height=180,
    header_background="#fff",
    header_color="#000",
)
template.main[0:2,0:6]=markdown
template.main[2:4,0:6]=years_figure
template.main[4:6,0:6]=species_figure

template.main[0:6,6:12]=mapbox
#template.main[3:6,6:]=heatmap
#template.main[6:9,0:12]=district_result
template.servable()